In [2]:
# imports
from bertopic import BERTopic
import pandas as pd
import spacy
import json
import os

In [73]:
# load book metadata dataframe
df_metadata = pd.read_csv("../data/interim/books/books_metadata.txt", dtype="str")
ids = list(df_metadata["id"])

In [74]:
# open a fulltext
id = "003"
load_path = "../data/interim/books/fulltexts/"
book_path = load_path + f"book{id}.txt"
with open(book_path) as file:
    fulltext = file.read()

In [68]:
# split into satisfactory chunks using spaCy
nlp = spacy.load("en_core_web_sm")
doc = nlp(fulltext)

chunk_lists = []
token_count = 0
chunk_list = []
for sent in doc.sents:
    l = len(sent)
    if token_count + l > 400:
        # 400 is an arbitrary value, keeping a large buffer
        # to keep BERT's tokens below 512
        chunk_lists.append(chunk_list)
        token_count = l
        chunk_list = []
        chunk_list.append(sent.text)
    else:
        token_count += l
        chunk_list.append(sent.text)
chunks = [" ".join(chunk_list) for chunk_list in chunk_lists]

In [69]:
# run BERTopic
model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")
df_fulltopic = model.get_topic_info()

In [70]:
topics, probabilities = model.transform(chunks)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches: 100%|██████████| 7/7 [00:15<00:00,  2.22s/it]


In [71]:
# extract topics and probabililties
topic_dicts = {}

for topic in topics:
    topic_dicts[topic] = {
        "prob": 0,
        "words": list(
            df_fulltopic.loc[df_fulltopic["Topic"] == topic]["Representation"]
        ),
    }

for i, topic in enumerate(topics):
    topic_dicts[topic]["prob"] += probabilities[i]

df_topic = pd.DataFrame.from_dict(topic_dicts, orient="index")
df_topic = df_topic.sort_values(by = 'prob', ascending=False)
df_topic

,prob,words
1471,7.567925,"[[legions, heresy, legion, crusade, factions, ..."
98,7.080056,"[[goddesses, stanzas, mythology, stanza, valky..."
2075,3.433458,"[[1605, gunpowder, undercroft, conspirators, 1..."
270,2.674975,"[[brigades, soldiers, reinforcements, troops, ..."
999,2.599750,"[[rescuers, rescuer, survivors, rescue, camped..."
...,...,...
1243,0.352430,"[[heterochromia, pigmentation, pigment, pigmen..."
769,0.351572,"[[verse, poetry, poet, poems, poem, poets, 166..."
273,0.349843,"[[refrigerants, refrigeration, refrigerant, re..."
1987,0.323385,"[[yoon, jin, jung, hye, kyung, ji, han, hyun, ..."


In [76]:
# full pipeline

failed_ids_tm = []

# directories
load_path = "../data/interim/books/fulltexts/"
save_path = "../data/interim/topics/"

# models
nlp = spacy.load("en_core_web_sm")
model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")
df_fulltopic = model.get_topic_info()

for id in ids[0:]:
    try:
        # open the fulltext
        book_path = load_path + f"book{id}.txt"
        with open(book_path) as file:
            fulltext = file.read()

        # split into satisfactory chunks using spaCy
        doc = nlp(fulltext)

        chunk_lists = []
        token_count = 0
        chunk_list = []
        for sent in doc.sents:
            l = len(sent)
            if token_count + l > 400:
                # 400 is an arbitrary value, keeping a large buffer
                # to keep BERT's tokens below 512
                chunk_lists.append(chunk_list)
                token_count = l
                chunk_list = []
                chunk_list.append(sent.text)
            else:
                token_count += l
                chunk_list.append(sent.text)
        chunks = [" ".join(chunk_list) for chunk_list in chunk_lists]

        # run topic modelling inference on each chunk
        topics, probabilities = model.transform(chunks)

        # extract topics and probabililties
        topic_dicts = {}

        for topic in topics:
            topic_dicts[topic] = {
                "prob": 0,
                "words": list(
                    df_fulltopic.loc[df_fulltopic["Topic"] == topic]["Representation"]
                ),
            }

        for i, topic in enumerate(topics):
            topic_dicts[topic]["prob"] += probabilities[i]

        df_topic = pd.DataFrame.from_dict(topic_dicts, orient="index")
        df_topic = df_topic.sort_values(by = 'prob', ascending=False)

        # write topic_df to file
        out_file = save_path + f"{id}.csv"
        df_topic.to_csv(out_file, index=False)
        print(f"ID:{id} done.")

    except:
        failed_ids_tm.append(id)
        print(f"ID:{id} failed.")

Batches: 100%|██████████| 2/2 [00:04<00:00,  2.10s/it]


ID:011 done.


Batches: 100%|██████████| 17/17 [00:50<00:00,  3.00s/it]


ID:012 done.


Batches: 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


...

Batches: 100%|██████████| 3/3 [00:04<00:00,  1.52s/it]


ID:099 done.


Batches: 100%|██████████| 9/9 [00:23<00:00,  2.57s/it]


ID:100 done.


Batches:   0%|          | 0/6 [00:00<?, ?it/s]